## Shell Contact Example

In [1]:
# !pip install debugpy

In [ ]:
import numpy as np

import dismech


geom = dismech.GeomParams(rod_r0=0.005,
                          shell_h=0.005)

material = dismech.Material(density=2000,
                            youngs_rod=0,
                            youngs_shell=1e4,
                            poisson_rod=0,
                            poisson_shell=0.5)

shell_contact_sim = dismech.SimParams(static_sim=False,
                                  two_d_sim=False,   # no twisting
                                  use_mid_edge=False,
                                  use_line_search=True,
                                  show_floor=False,
                                  log_data=True,
                                  log_step=1,
                                  dt=1e-2,
                                  max_iter=120,
                                  total_time=0.18,
                                  plot_step=1,
                                  tol=1e-4,
                                  ftol=1e-4,
                                  dtol=1e-2)

env = dismech.Environment()
env.add_force('gravity', g=np.array([0.0, 0.0, -9.81]))
# env.add_force('floorContact', ground_z=-1e-3, stiffness=20, delta=1e-3, h=1e-3)
env.add_force('selfContact', delta=1e-5, h=1e-4, kc=0.01)
env.add_force('damping', eta = 0.01)
#env.set_static()   

geo = dismech.Geometry.from_txt('input_shell_more_curved.txt')
# geo = dismech.Geometry.from_txt('input_oneTriangle.txt')
# geo = dismech.Geometry.from_txt('input_twoTriangleContact_p2t.txt')

robot = dismech.SoftRobot(geom, material, geo, shell_contact_sim, env)

# fixed_points = np.array(np.where(
#     (robot.state.q[:robot.end_node_dof_index].reshape(-1, 3)[:, 0] <= 0.01) &
#     (robot.state.q[:robot.end_node_dof_index].reshape(-1, 3)[:, 2] == 0.0)
# )[0])
fixed_points = np.array(np.where(
    (robot.state.q[:robot.end_node_dof_index].reshape(-1, 3)[:, 2] < 0.001)
)[0])


robot = robot.fix_nodes(fixed_points)

# u = robot.state.u
# u[11] = -0.5
# u[14] = -0.5
# u[17] = -0.5

### Time Stepping

As we are performing a static simulation, we must change gravity for each time step.

In [3]:
stepper = dismech.ImplicitEulerTimeStepper(robot)

robots = stepper.simulate()

qs = np.stack([robot.state.q for robot in robots])

delta: 0.1
h: 1.0
K1: 150.0
scale:  1000.0
kc:  1
upper limit for quadratic: 1.9
upper limit for smooth: 2.1
0.014286235227043897
0.0008236845079003806
current_time:  0.01
0.020404860257272256
0.0016177278521745863
current_time:  0.02
0.023623102569537827
0.001974971149710432
current_time:  0.03
0.025100415811233266
0.0019941125261025952
current_time:  0.04
0.02554719238426611
0.0018348238465889174
current_time:  0.05
0.025425883387026952
0.0016151256551377564
current_time:  0.06
0.025031528804816683
0.0013950452805491084
current_time:  0.07
0.024541055624970606
0.0012014014732999144
current_time:  0.08
0.024049981200323232
0.0010799686903259624
current_time:  0.09
0.023602808326518347
0.001078254971521092
current_time:  0.1
0.023216155930493493
0.0011822602817904363
current_time:  0.11
0.022893540815031508
0.0013313755536222857
current_time:  0.12
0.022633370313146103
0.0014765644409748281
current_time:  0.13
0.02243266271759794
0.001596982697217991
current_time:  0.14
0.0222884403772

KeyboardInterrupt: 

In [ ]:
t = np.arange(robot.sim_params.total_time, step=robot.sim_params.dt)
options = dismech.AnimationOptions(title='Shell contact p2p', plot_step=1)

fig = dismech.get_interactive_animation_plotly(robot, t, qs, options)
fig.show()

In [ ]:
#ani = dismech.get_animation(robot, t, qs, options)
#ani.save('5e-5.gif')